In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this(by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Preprocessing the input data

In [ ]:
images_path = '/kaggle/input/tattoo-test-images'
labels_path = '/kaggle/input/resized-tattoo-labels'

In [ ]:
images_lst = os.listdir(images_path)

In [ ]:
bb = pd.read_csv('/kaggle/input/resized-tattoo-labels/resized_IMG_0998.txt', columns=['classs', 'xmin', 'xmax', 'ymin', 'ymax'])
bb

In [ ]:
img_bbox = pd.read-csv(img_label)

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant. 

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

## Install Segment Anything Model (SAM) and other dependencies

In [ ]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

In [ ]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

### Download SAM weights

In [ ]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

## Pre-process dataset

**NONE:** Let's download few example images. Feel free to use your images or videos.

In [ ]:
dataset = '/kaggle/input/tattoo-test-dataset'

imgs = os.listdir(dataset)
imgs

## Load Model

In [ ]:
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor



def load_SAM_predictor():

    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    MODEL_TYPE = "vit_h"

    sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
    
    mask_predictor = SamPredictor(sam)
    
    return mask_predictor

`SamAutomaticMaskGenerator` returns a `list` of masks, where each mask is a `dict` containing various information about the mask:

* `segmentation` - `[np.ndarray]` - the mask with `(W, H)` shape, and `bool` type
* `area` - `[int]` - the area of the mask in pixels
* `bbox` - `[List[int]]` - the boundary box of the mask in `xywh` format
* `predicted_iou` - `[float]` - the model's own prediction for the quality of the mask
* `point_coords` - `[List[List[float]]]` - the sampled input point that generated this mask
* `stability_score` - `[float]` - an additional measure of mask quality
* `crop_box` - `List[int]` - the crop of the image used to generate this mask in `xywh` format

## Generate Segmentation with Bounding Box

The `SamPredictor` class provides an easy interface to the model for prompting the model. It allows the user to first set an image using the `set_image` method, which calculates the necessary image embeddings. Then, prompts can be provided via the `predict` method to efficiently predict masks from those prompts. The model can take as input both point and box prompts, as well as masks from the previous iteration of prediction.

In [ ]:
# # helper function that loads an image before adding it to the widget

# import base64

# def encode_image(filepath):
#     with open(filepath, 'rb') as f:
#         image_bytes = f.read()
#     encoded = str(base64.b64encode(image_bytes), 'utf-8')
#     return "data:image/jpg;base64,"+encoded

**NOTE:** Execute cell below and use your mouse to draw bounding box on the image 👇

In [ ]:
# IS_COLAB = False

# if IS_COLAB:
#     from google.colab import output
#     output.enable_custom_widget_manager()

# from jupyter_bbox_widget import BBoxWidget

# widget = BBoxWidget()
# widget.image = encode_image(IMAGE_PATH)
# widget

In [ ]:
# widget.bboxes

### Generate masks with SAM

**NOTE:** `SamPredictor.predict` method takes `np.ndarray` `box` argument in `[x_min, y_min, x_max, y_max]` format. Let's reorganise your data first

In [ ]:


def yolo_to_coco(center_x, center_y, bbox_width, bbox_height, width, height):
    x_min = int((center_x - bbox_width/2) * width)
    y_min = int((center_y - bbox_height/2) * height)
    coco_width = int(bbox_width * width)
    coco_height = int(bbox_height * height)
    
    return x_min, y_min, coco_width, coco_height


def get_bbox(a, b, aa, bb, image_width, image_height):
    
    x,y, xx, yy =  yolo_to_coco(a, b, aa, bb, image_width, image_height)
    
    # default_box is going to be used if you will not draw any box on image above
    default_box = {'x': x, 'y': y, 'width': xx, 'height': yy, 'label': ''}

    # box = widget.bboxes[0] if widget.bboxes else default_box
    box = default_box
    box = np.array([
        box['x'], 
        box['y'], 
        box['x'] + box['width'], 
        box['y'] + box['height']
    ])
    return box

In [ ]:


def predict_mask(IMAGE_PATH, mask_predictor, box):


    image_bgr = cv2.imread(IMAGE_PATH)
#     image_bgr = cv2.resize(image_bgr, (416, 416))
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    mask_predictor.set_image(image_rgb)

    masks, scores, logits = mask_predictor.predict(
        box=box,
        multimask_output=True
    )
    return masks, scores, logits

### Results visualisation with Supervision

In [ ]:
def visualize_predicted_masks(mask_par, box, IMAGE_PATH):

    image_bgr = cv2.imread(IMAGE_PATH)
#     image_bgr = cv2.resize(image_bgr, (416, 416))
    
    box_annotator = sv.BoxAnnotator(color=sv.Color.red())
    mask_annotator = sv.MaskAnnotator(color=sv.Color.red())

    detections = sv.Detections(
        xyxy=sv.mask_to_xyxy(masks=masks),
        mask=masks
    )
    detections = detections[detections.area == np.max(detections.area)]

    source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections, skip_label=True)
    segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

    sv.plot_images_grid(
        images=[source_image, segmented_image],
        grid_size=(1, 2),
        titles=['source image', 'segmented image']
    )
    return segmented_image

In [ ]:
# import supervision as v

# sv.plot_images_grid(
#     images=masks,
#     grid_size=(1, 4),
#     size=(16, 4)
# )

## Remove Background

In [ ]:
import cv2

def extract_tattoo(IMAGE_PATH, segmented_image, mask_value):

    original_image = cv2.imread(IMAGE_PATH)
#     original_image = cv2.resize(original_image, (416, 416))

    segmented_image = segmented_image

    # Load the mask value for the segmented image
    mask_value = mask_value

    # Create a new image of the same size as the original image
    extracted_image = np.zeros_like(original_image)

    # Copy the pixels from the original image to the new image, but only for the areas where the mask value is not zero
    extracted_image[np.where(mask_value != False)] = original_image[np.where(mask_value != False)]

    # Save the extracted portion of the image as a new image file
#     cv2.imwrite("extracted_image.jpg", extracted_image)
    
    return extracted_image


In [ ]:
def main(IMAGE_DATA):
    
    IMAGE_PATH = IMAGE_DATA['IMAGE_PATH']
    IMAGE_BBOX = IMAGE_DATA['IMAGE_BBOX']
    image_width = IMAGE_DATA['WIDTH']
    image_height = IMAGE_DATA['HEIGHT']
    mask_index = IMAGE_DATA['MASK_INDEX']
    
    mask_predictor = load_SAM_predictor()
    box = get_bbox(IMAGE_BBOX[0], IMAGE_BBOX[1],
                   IMAGE_BBOX[2], IMAGE_BBOX[3], image_width, image_height)
    
    masks, scores, logits = predict_mask(IMAGE_PATH, mask_predictor, box)
    segmented_image = visualize_predicted_masks(masks, box, IMAGE_PATH)
    
    mask_value = masks[mask_index]
    
    tattoo = extract_tattoo(IMAGE_PATH, segmented_image, mask_value)
    
    tattoo = cv2.cvtColor(tattoo, cv2.COLOR_BGR2RGB)
    plt.imshow(tattoo)
    tattoo.shape
    return tattoo

In [ ]:
import cv2
import numpy as np
import supervision as sv
import numpy as np

# IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)
# IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/WhatsApp Image 2022-11-20 at 19.08.20 (4).jpeg'


In [ ]:


# IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/IMG_1004.jpg'
# IMAGE_BBOX = [0.507212, 0.600962, 0.144231, 0.149038]

# WIDTH = 416
# HEIGHT = 416
# MASK_INDEX = 0

# IMAGE_DATA = {'IMAGE_PATH': IMAGE_PATH,
#               'IMAGE_BBOX': IMAGE_BBOX, 'WIDTH': WIDTH,
#               'HEIGHT': HEIGHT, 'MASK_INDEX': MASK_INDEX}

# main(IMAGE_DATA)

In [ ]:
def get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name):

#     IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/IMG_1004.jpg'
#     IMAGE_BBOX = [0.507212, 0.600962, 0.144231, 0.149038]

    WIDTH = 416
    HEIGHT = 416
    MASK_INDEX = 0

    IMAGE_DATA = {'IMAGE_PATH': IMAGE_PATH,
                  'IMAGE_BBOX': IMAGE_BBOX, 'WIDTH': WIDTH,
                  'HEIGHT': HEIGHT, 'MASK_INDEX': MASK_INDEX}

    tattoo = main(IMAGE_DATA)
    cv2.imwrite(f"/kaggle/working/tattoo_images/{image_name}", tattoo)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_andrea10.jpg"

image_name = "resized_andrea10.jpg"

MASK_INDEX = 0

IMAGE_BBOX = "0.481971 0.479567 0.420673 0.329327"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
# !mkdir /kaggle/working/tattoo_images


In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_andrea10.jpg"

image_name = "resized_andrea10.jpg"

MASK_INDEX = 0

IMAGE_BBOX = "0.481971 0.479567 0.420673 0.329327"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_1008.jpg"

image_name = "resized_IMG_1008.jpg"

MASK_INDEX = 0

IMAGE_BBOX = "0.4375 0.415865 0.389423 0.370192"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_1012.jpg"

image_name = "resized_IMG_1012.jpg"

MASK_INDEX = 2

IMAGE_BBOX = "0.47476 0.519231 0.598558 0.625"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_2486.jpg"

image_name = "resized_IMG_2486.jpg"

MASK_INDEX = 2

IMAGE_BBOX = "0.575721 0.514423 0.848558 0.283654"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_2487.jpg"

image_name = "resized_IMG_2487.jpg"

MASK_INDEX = 2

IMAGE_BBOX = "0.501202 0.448317 0.151442 0.266827"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_2495.jpg"

image_name = "resized_IMG_2495.jpg"

MASK_INDEX = 2

IMAGE_BBOX = "0.501202 0.61899 0.632212 0.603365"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_2497.jpg"

image_name = "resized_IMG_2497.jpg"

MASK_INDEX = 2

IMAGE_BBOX = "0.503606 0.501202 0.757212 0.930288"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_2496.jpg"

image_name = "resized_IMG_2496.jpg"

MASK_INDEX = 2

IMAGE_BBOX = "0.5 0.584135 0.730769 0.740385"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_WhatsApp Image 2022-11-21 at 12.08.46 (1).jpeg"

image_name = "resized_WhatsApp Image 2022-11-21 at 12.08.46 (1).jpeg"

MASK_INDEX = 0

IMAGE_BBOX = "0.501202 0.481971 0.757212 0.310096"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_WhatsApp Image 2022-11-20 at 19.41.07 (6).jpeg"

image_name = "resized_WhatsApp Image 2022-11-20 at 19.41.07 (6).jpeg"

MASK_INDEX = 2

IMAGE_BBOX = "0.450721 0.408654 0.78125 0.778846"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_WhatsApp Image 2022-11-20 at 19.08.19.jpeg"

image_name = "resized_WhatsApp Image 2022-11-20 at 19.08.19.jpeg"

MASK_INDEX = 2

IMAGE_BBOX = "0.503606 0.402644 0.319712 0.175481"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_WhatsApp Image 2022-11-20 at 19.41.05 (2).jpeg"

image_name = "resized_WhatsApp Image 2022-11-20 at 19.41.05 (2).jpeg"

MASK_INDEX = 0

IMAGE_BBOX = "0.47476 0.590144 0.771635 0.709135"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_WhatsApp Image 2022-11-20 at 19.08.20 (7).jpeg"

image_name = "resized_WhatsApp Image 2022-11-20 at 19.08.20 (7).jpeg"

MASK_INDEX = 0

IMAGE_BBOX = "0.78726 0.384615 0.324519 0.557692"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_WhatsApp Image 2022-11-20 at 19.08.21 (2).jpeg"

image_name = "resized_WhatsApp Image 2022-11-20 at 19.08.21 (2).jpeg"

MASK_INDEX = 0

IMAGE_BBOX = "0.504808 0.479567 0.394231 0.266827"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:

IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_WhatsApp Image 2022-11-20 at 19.08.20 (5).jpeg"

image_name = "resized_WhatsApp Image 2022-11-20 at 19.08.20 (5).jpeg"

MASK_INDEX = 0

IMAGE_BBOX = "0.359375 0.492788 0.473558 0.259615"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:

IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_siuky5.jpg"

image_name = "resized_siuky5.jpg"

MASK_INDEX = 0

IMAGE_BBOX = "0.484375 0.5 0.536058 0.572115"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:

IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_4482.jpg"

image_name = "resized_IMG_4482.jpg"

MASK_INDEX = 0

IMAGE_BBOX = "0.513221 0.449519 0.71875 0.894231"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
IMAGE_PATH = "/kaggle/input/resized-tattoo-images/resized_IMG_4461.jpg"

image_name = "resized_IMG_4461.jpg"

MASK_INDEX = 0

IMAGE_BBOX = "0.491587 0.486779 0.834135 0.685096"

IMAGE_BBOX = [float(i) for i in IMAGE_BBOX.split()]


get_tattoo(IMAGE_PATH, IMAGE_BBOX, MASK_INDEX, image_name)

In [ ]:
!mkdir /kaggle/working/new

In [ ]:
!cp -r "/kaggle/working/tattoo_images" "/kaggle/working/new"

In [ ]:
os.listdir('/kaggle/working/tattoo_images')

In [ ]:
IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/IMG_2701.jpg'
IMAGE_BBOX = [0.483173, 0.350962, 0.947115, 0.230769]

WIDTH = 416
HEIGHT = 416
MASK_INDEX = 2

IMAGE_DATA = {'IMAGE_PATH': IMAGE_PATH,
              'IMAGE_BBOX': IMAGE_BBOX, 'WIDTH': WIDTH,
              'HEIGHT': HEIGHT, 'MASK_INDEX': MASK_INDEX}

main(IMAGE_DATA)

In [ ]:

IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/IMG_4474.jpg'
IMAGE_BBOX = [0.373798, 0.802885, 0.536058, 0.298077]

WIDTH = 416
HEIGHT = 416
MASK_INDEX = 0

IMAGE_DATA = {'IMAGE_PATH': IMAGE_PATH,
              'IMAGE_BBOX': IMAGE_BBOX, 'WIDTH': WIDTH,
              'HEIGHT': HEIGHT, 'MASK_INDEX': MASK_INDEX}

main(IMAGE_DATA)

In [ ]:

IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/toan-nguyen-gSO-66i1ZnI-unsplash.JPG'
IMAGE_BBOX = [0.486779, 0.620192, 0.199519, 0.278846]

WIDTH = 416
HEIGHT = 416
MASK_INDEX = 2

IMAGE_DATA = {'IMAGE_PATH': IMAGE_PATH,
              'IMAGE_BBOX': IMAGE_BBOX, 'WIDTH': WIDTH,
              'HEIGHT': HEIGHT, 'MASK_INDEX': MASK_INDEX}

main(IMAGE_DATA)

In [ ]:


IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/WhatsApp Image 2022-11-20 at 19.08.21 (1).jpeg'
IMAGE_BBOX = [0.519231, 0.509615, 0.543269, 0.298077]

WIDTH = 416
HEIGHT = 416
MASK_INDEX = 2

IMAGE_DATA = {'IMAGE_PATH': IMAGE_PATH,
              'IMAGE_BBOX': IMAGE_BBOX, 'WIDTH': WIDTH,
              'HEIGHT': HEIGHT, 'MASK_INDEX': MASK_INDEX}

main(IMAGE_DATA)

In [ ]:


IMAGE_PATH = '/kaggle/input/small-tattoo-dataset/WhatsApp Image 2022-11-20 at 19.08.21 (1).jpeg'
IMAGE_BBOX = [0.441106 0.507212 0.194712, 0.235577]

WIDTH = 416
HEIGHT = 416
MASK_INDEX = 2

IMAGE_DATA = {'IMAGE_PATH': IMAGE_PATH,
              'IMAGE_BBOX': IMAGE_BBOX, 'WIDTH': WIDTH,
              'HEIGHT': HEIGHT, 'MASK_INDEX': MASK_INDEX}

main(IMAGE_DATA)

In [ ]:
# a = 0.512019
# b = 0.564904
# aa = 0.322115
# bb = 0.533654


# a, b, aa, bb = 0.481971, 0.433894, 0.353365, 0.257212

# a, b, aa, bb = 0.5125, 0.603437, 0.158333, 0.143125


# 1
a, b, aa, bb, = 0.507212, 0.600962, 0.144231, 0.149038


## BGR

# Seg

## Segment Anything in Roboflow Universe Dataset

### Utils Supporting Dataset Processing

A couple of helper functions that, unfortunately, we have to write ourselves to facilitate the processing of COCO annotations.

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Union, Optional
from dataclasses_json import dataclass_json
from supervision import Detections


@dataclass_json
@dataclass
class COCOCategory:
    id: int
    name: str
    supercategory: str


@dataclass_json
@dataclass
class COCOImage:
    id: int
    width: int
    height: int
    file_name: str
    license: int
    date_captured: str
    coco_url: Optional[str] = None
    flickr_url: Optional[str] = None


@dataclass_json
@dataclass
class COCOAnnotation:
    id: int
    image_id: int
    category_id: int
    segmentation: List[List[float]]
    area: float
    bbox: Tuple[float, float, float, float]
    iscrowd: int


@dataclass_json
@dataclass
class COCOLicense:
    id: int
    name: str
    url: str


@dataclass_json
@dataclass
class COCOJson:
    images: List[COCOImage]
    annotations: List[COCOAnnotation]
    categories: List[COCOCategory]
    licenses: List[COCOLicense]


def load_coco_json(json_file: str) -> COCOJson:
    import json

    with open(json_file, "r") as f:
        json_data = json.load(f)

    return COCOJson.from_dict(json_data)


class COCOJsonUtility:
    @staticmethod
    def get_annotations_by_image_id(coco_data: COCOJson, image_id: int) -> List[COCOAnnotation]:
        return [annotation for annotation in coco_data.annotations if annotation.image_id == image_id]

    @staticmethod
    def get_annotations_by_image_path(coco_data: COCOJson, image_path: str) -> Optional[List[COCOAnnotation]]:
        image = COCOJsonUtility.get_image_by_path(coco_data, image_path)
        if image:
            return COCOJsonUtility.get_annotations_by_image_id(coco_data, image.id)
        else:
            return None

    @staticmethod
    def get_image_by_path(coco_data: COCOJson, image_path: str) -> Optional[COCOImage]:
        for image in coco_data.images:
            if image.file_name == image_path:
                return image
        return None

    @staticmethod
    def annotations2detections(annotations: List[COCOAnnotation]) -> Detections:
        class_id, xyxy = [], []

        for annotation in annotations:
            x_min, y_min, width, height = annotation.bbox
            class_id.append(annotation.category_id)
            xyxy.append([
                x_min,
                y_min,
                x_min + width,
                y_min + height
            ])

        return Detections(
            xyxy=np.array(xyxy, dtype=int),
            class_id=np.array(class_id, dtype=int)
        )

### Download Dataset from Roboflow

In [ ]:
%cd {HOME}

import roboflow
from roboflow import Roboflow

# roboflow.login()

# rf = Roboflow()

# project = rf.workspace("hashira-fhxpj").project("mri-brain-tumor")
# dataset = project.version(1).download("coco")


from roboflow import Roboflow
rf = Roboflow(api_key="1Pt0GpqOFPYhcRmLYYm3")
project = rf.workspace("octa-cube").project("tatto-detection-pjkn9")
dataset = project.version(1).download("yolov5")

In [ ]:
os.listdir('/kaggle/working/Tatto-detection-1')

In [ ]:
dataset.location

In [ ]:
import os

DATA_SET_SUBDIRECTORY = "test"
ANNOTATIONS_FILE_NAME = "README.roboflow.txt"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, ANNOTATIONS_FILE_NAME)

In [ ]:
coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

CLASSES = [
    category.name
    for category
    in coco_data.categories
    if category.supercategory != 'none'
]

IMAGES = [
    image.file_name
    for image
    in coco_data.images
]

In [ ]:
CLASSES

### Single Image Bounding Box to Mask

In [ ]:
# set random seed to allow easy reproduction of the experiment

import random
random.seed(10)

In [ ]:
EXAMPLE_IMAGE_NAME = random.choice(IMAGES)
EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)

# load dataset annotations
annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)

# small hack - coco numerate classes from 1, model from 0 + we drop first redundant class from coco json
ground_truth.class_id = ground_truth.class_id - 1

# load image
image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

# initiate annotator
box_annotator = sv.BoxAnnotator(color=sv.Color.red())
mask_annotator = sv.MaskAnnotator(color=sv.Color.red())

# annotate ground truth
annotated_frame_ground_truth = box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth, skip_label=True)

# run SAM inference
mask_predictor.set_image(image_rgb)

masks, scores, logits = mask_predictor.predict(
    box=ground_truth.xyxy[0],
    multimask_output=True
)

detections = sv.Detections(
    xyxy=sv.mask_to_xyxy(masks=masks),
    mask=masks
)
detections = detections[detections.area == np.max(detections.area)]

annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[annotated_frame_ground_truth, annotated_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)

## 🏆 Congratulations

### Learning Resources

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.